## LLM + Code RAG 
This notebook uses FAISS as the vectorDB to store and retreive similar questions from the example databses to provide few shot learning examples

In [1]:
# !pip install faiss-cpu

In [73]:
%load_ext autoreload
%autoreload 2

In [4]:
import jupyter_black

jupyter_black.load()

In [75]:
import pandas as pd
import os

from tqdm import tqdm

tqdm.pandas()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable, RunnableParallel

# from langchain.chains import create_sql_query_chain

from config import *
from utils import *

from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS

## Load Golden Examples (assuming I take 10 of corrected SQL)

In [33]:
# Load input data
df_qa = pd.read_csv(QA_DATA_PATH)

# Remove all gpt4 columns
gpt4_cols = [col for col in df_qa.columns if "gpt4" in col]
df_qa.drop(columns=gpt4_cols, inplace=True)
print(df_qa.shape)
df_qa.head()

(40, 10)


qa_id                                 question  \
0    t1          count the number of outpatients   
1    t2           count the number of inpatients   
2    t3       select all inpatients in year 2005   
3    t4      select all outpatients in year 2006   
4    t5  return total patients from 2015 to 2016   

                        question_with_list  \
0          count the number of outpatients   
1           count the number of inpatients   
2       select all inpatients in year 2005   
3      select all outpatients in year 2006   
4  return total patients from 2015 to 2016   

                                               query  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                                     query_corrected  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                           query_corrected_with_list  difficulty_level  \
0  select count(distinct desynpuf_id) from rwdex_...                 1   
1  select count(distinct desynpuf_id) from rwdex_...                 1   
2  select distinct desynpuf_id from rwdex_raw_syn...                 2   
3  select distinct desynpuf_id from rwdex_raw_syn...                 2   
4  select distinct desynpuf_id from rwdex_raw_syn...                 3   

  code_list_needed code_list_table  query_corrected_with_list_executed  
0              NaN             NaN                                True  
1              NaN             NaN                                True  
2              Yes             NaN                                True  
3              Yes             NaN                                True  
4              NaN             NaN                                True

In [35]:
sample_df = df_qa[df_qa['query_corrected_with_list_executed']==True][["question", "query_corrected_with_list"]].sample(n=10, random_state=1)
sample_df.shape

(10, 2)

In [41]:
import pickle

sample_dict = {key: val for _, (key, val) in sample_df.iterrows()}

with open("../faiss_index/faiss_examples.pickle", "wb") as file:
    pickle.dump(sample_dict, file)

## Create question embeddings and store to FAISS

In [42]:
embeddings = BedrockEmbeddings(endpoint_url='https://bedrock.us-east-1.amazonaws.com',
                               model_id = 'amazon.titan-embed-g1-text-02',
                               region_name = 'us-east-1')

In [11]:
question_embeddings = embeddings.embed_documents(list(sample_df.question))

In [47]:
db = FAISS.from_texts(list(sample_df.question), embeddings)
db.save_local("../faiss_index/faiss_index")

## Load FAISS vectorDB to query similar questions for few shots learning

In [107]:
golden_examples.keys()

dict_keys(['select patients who received therapy listed in prcdr_code_list', 'select inpatients who are in hospital for at least 30 days', 'select all female outpatients', 'select the inpatients who have died with diagnosis of diseases listed in code_list', 'select inpatients with the number of distinct primary diagnosis', 'select the earliest diagnosis date of disease in code_list for inpatients', "select inpatients who received drug with code 'xxx' after 2015", 'select all outpatients in year 2006', 'select California inpatients with diagnosis of disease in code_list', 'select white inpatients'])

In [53]:
new_db = FAISS.load_local("../faiss_index/faiss_index", embeddings)
retriever = new_db.as_retriever(search_kwargs={"k":3})

with open("../faiss_index/faiss_examples.pickle", "rb") as file:
    golden_examples = pickle.load(file)

In [81]:
# Get tables info with sample rows
tables_info = get_db_tables_info()

print(tables_info[:200])

Tables Info already exists! Loading.

CREATE TABLE rwdex_raw_synpuf.beneficiary_summary (
	desynpuf_id VARCHAR(256), 
	bene_birth_dt DATE, 
	bene_death_dt DATE, 
	bene_sex_ident_cd INTEGER, 
	bene_race_cd INTEGER, 
	bene_esrd_ind VARCHAR


In [82]:
# Get column descriptions in xml format
col_descriptions = get_column_descriptions()
print(col_descriptions[:400])

<data>
  <row>
    <column_name>DESYNPUF_ID</column_name>
    <column_description>Beneficiary Code</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_BIRTH_DT</column_name>
    <column_description>Date of birth</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_DEATH_DT</column_name>


In [78]:
# Create LLM
llm = get_bedrock_llm(model_id=MODEL_ID, verbose=True, model_config=MODEL_CONFIG)

# Create Prompt from template
prompt = PromptTemplate.from_template(PROMPT)

# Create DB chain
db_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

In [79]:
prompt

PromptTemplate(input_variables=['columns_description', 'db_schema_name', 'query_examples', 'question', 'table_info'], template='\n\nHuman: You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. Database schema is provided in <table-info> tag and input question is in <question> tag. Columns description is also provided within <columns-description> tag. In addition, the database schema name is also added in <schema-name> tag.\n\nProvide only the necessary answer. When

In [98]:
# Let's try with the first query
question = "select the inpatients who receive hospitalization because of diabetes of code list X"  # df_qa.iloc[0]["question_with_list"]
# question = "select the outpatients who are diagnosed with disease in code list X and are sent to hospital thereafter"
# question = "select the outpatients who are diagnosed with diabetes of code list X"

def build_few_shot_examples(question, retriever):
    similar_questions = [content.page_content for content in retriever.get_relevant_documents(question)]
    few_shot_sql = [golden_examples[question] for question in similar_questions]
    
    output = ""
    
    for q,s in zip(similar_questions, few_shot_sql):
        output += f"<question>{q}</question>"
        output += f"\n<sql>\n{s}\n</sql>\n\n"
        
    return output

few_shot_examples = build_few_shot_examples(question, retriever)
print(few_shot_examples)

<question>select California inpatients with diagnosis of disease in code_list</question>
<sql>
select distinct a.desynpuf_id from rwdex_raw_synpuf.inpatient_claims a inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id and b.sp_state_code = 12 and a.admtng_icd9_dgns_cd in (25000, 25001, 25002, 25003, 25010, 25011, 25012, 25013, 25020, 25021, 25022, 25023, 25030, 25031, 25032, 25033, 25040, 25041, 25042, 25043, 25050, 25051, 25052, 25053, 25060, 25061, 25062, 25063, 25070, 25071, 25072, 25073, 25080, 25081, 25082, 25083, 25090, 25091, 25092, 25093)
</sql>

<question>select the inpatients who have died with diagnosis of diseases listed in code_list</question>
<sql>
select distinct a.desynpuf_id from rwdex_raw_synpuf.inpatient_claims a inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id and b.bene_death_dt is not null and a.admtng_icd9_dgns_cd in (25000, 25001, 25002, 25003, 25010, 25011, 25012, 25013, 25020, 25021, 25022, 25023

In [99]:
inputs = {
    "question": question,
    "db_schema_name": DB_SCHEMA_NAME,
    "table_info": tables_info,
    "columns_description": col_descriptions,
    "query_examples": few_shot_examples
}

response = db_chain.run(inputs)
print(response)

 <sql>
select distinct a.desynpuf_id 
from rwdex_raw_synpuf.inpatient_claims a
inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id
where a.admtng_icd9_dgns_cd in (CODE_LIST_X)
</sql>

<summary>
The SQL query selects distinct beneficiary IDs (DESYNPUF_ID) from the inpatient_claims table who have a diabetes diagnosis code (admtng_icd9_dgns_cd) in the given code list (CODE_LIST_X). It joins with the beneficiary_summary table on DESYNPUF_ID to get additional beneficiary information if needed. This will return the inpatients that were hospitalized due to diabetes, based on the diagnosis codes.
</summary>


In [100]:
sql = extract_xml(tags=["sql"], text=response)
summary = extract_xml(tags=["summary"], text=response)

In [101]:
print(sql)

select distinct a.desynpuf_id 
from rwdex_raw_synpuf.inpatient_claims a
inner join rwdex_raw_synpuf.beneficiary_summary b on a.desynpuf_id = b.desynpuf_id
where a.admtng_icd9_dgns_cd in (CODE_LIST_X)


In [102]:
print(summary)

The SQL query selects distinct beneficiary IDs (DESYNPUF_ID) from the inpatient_claims table who have a diabetes diagnosis code (admtng_icd9_dgns_cd) in the given code list (CODE_LIST_X). It joins with the beneficiary_summary table on DESYNPUF_ID to get additional beneficiary information if needed. This will return the inpatients that were hospitalized due to diabetes, based on the diagnosis codes.


## Add new examples (if user specifies)

In [115]:
new_example = {"count the number of outpatients": "select count(distinct desynpuf_id) from rwdex_raw_synpuf.outpatient_claims;"}

# Save to FAISS
new_db.add_texts(list(new_example.keys()))
new_db.save_local("../faiss_index/faiss_index")

# Save to pickle file
golden_examples.update(new_example)
with open("../faiss_index/faiss_examples.pickle", "wb") as file:
    pickle.dump(golden_examples, file)